In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms

In [ ]:
# 1. Configuration and Data Loading
BATCH_SIZE = 4
transform = transforms.Compose([
    transforms.ToTensor(),
])

In [ ]:
# load data
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [ ]:
# Use GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 2. Define the Neural Network Modelclass SimpleNN(nn.Module):

In [ ]:
model = ().to(device)

In [ ]:
# 3. Define Loss and Optimizer
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# 4. Training Loop
NUM_EPOCHS = 5

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()  # Clear gradients
        output = model(data)  # Forward pass
        loss = criterion(output, target)  # Calculate loss
        loss.backward()  # Backward pass (calculate gradients)
        optimizer.step()  # Update weights

        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch + 1}/{NUM_EPOCHS} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

In [ ]:
# 5. Testing the Model
model.eval()
test_loss = 0
correct = 0

In [ ]:
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

In [ ]:
test_loss /= len(test_loader.dataset)
accuracy = 100. * correct / len(test_loader.dataset)

print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
      f'({accuracy:.2f}%)\n')

# 6. Save the model (optional)
torch.save(model.state_dict(), "students_simple_nn.pth")